## All COVID-19 Cases by Country

In [25]:
import pandas as pd
import numpy as np
import os

In [26]:
# Specific to JHCSSE data
def clean_data(file):
    df = pd.read_csv(file)
    df = df.drop(['Lat','Long','Province/State'], axis=1)
    df.rename(columns={'Country/Region': 'Country_Region'}, inplace = True)
    df = df.groupby('Country_Region').sum()
    df.rename(index={'US':'United States',
        'Taiwan*':'Taiwan',
        'Korea, South':'South Korea',
        'Congo (Brazzaville)':'Congo',
        'Congo (Kinshasa)':'Democratic Republic of Congo',
        'Czechia':'Czech Republic',
        'Gambia, The':'Gambia',},inplace=True)
    return df

#### Confirmed Cases

In [27]:
# All Countries
confirmed_df = clean_data('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
confirmed_df.head(1)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,273,281,299,349,367,423,444,484,521,555


In [38]:
# Global row
global_row = pd.Series(confirmed_df.sum(), name='Global')
confirmed_df = confirmed_df.append(global_row)
confirmed_df.tail()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20
Country_Region,,,,,,,,,,,,,,,,,,,,,
Western Sahara,0,0,0,0,0,0,0,0,0,0,...,0,0,0,4,4,4,4,4,4,4
Yemen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
Zambia,0,0,0,0,0,0,0,0,0,0,...,39,39,39,39,39,39,39,39,40,40
Zimbabwe,0,0,0,0,0,0,0,0,0,0,...,9,9,9,9,10,11,11,11,13,14
Global,555,654,941,1434,2118,2927,5578,6166,8234,9927,...,1013466,1095917,1197408,1272115,1345101,1426096,1511104,1595350,1691719,1771514


In [51]:
# Top Countries
n = 9
last_day = confirmed_df.columns[-1]
top_confirmed_df = confirmed_df.nlargest(n, last_day)
top_confirmed_df.drop(index='Global', inplace=True)
top_confirmed_df.index

Index(['Global', 'United States', 'Spain', 'Italy', 'France', 'Germany',
       'China', 'United Kingdom', 'Iran'],
      dtype='object', name='Country_Region')

#### Fatalities

In [53]:
# All Countries
fatalities_df = clean_data('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

# Global row
global_row = pd.Series(fatalities_df.sum(), name='Global')
fatalities_df = fatalities_df.append(global_row)

# Top Countries
top_fatalities_df = fatalities_df.loc[fatalities_df.index.isin(top_confirmed_df.index)]

#### Recovered

In [54]:
# All Countries
recovered_df = clean_data('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

# Global row
global_row = pd.Series(recovered_df.sum(), name='Global')
recovered_df = recovered_df.append(global_row)

# Top Countries
top_recovered_df = recovered_df.loc[recovered_df.index.isin(top_confirmed_df.index)]

#### Infected = Confirmed - Recovered

In [55]:
# All Countries
infected_df = confirmed_df-recovered_df

# Global row
global_row = pd.Series(infected_df.sum(), name='Global')
infected_df = infected_df.append(global_row)

# Top Countries
top_infected_df = infected_df.loc[infected_df.index.isin(top_confirmed_df.index)]

#### Most recent

In [56]:
def recent_stat(country_name, df):
    lookup = list(df.loc[df.index == country_name].values[0])
    lookup.reverse()
    most_recent = next((i for i in lookup if i.any()), None)
    return most_recent

# Example:
# recent_stat("Afghanistan", fatalities_df)

#### All Cases

In [57]:
countries = confirmed_df.index
all_df = pd.DataFrame(columns=['Country_Region', 'Confirmed', 'Recovered', 'Fatalities'], data=[])
for country in countries:
    all_df = all_df.append({'Country_Region': country,
                            'Confirmed': recent_stat(country, confirmed_df),
                            'Recovered': recent_stat(country, recovered_df),
                            'Fatalities': recent_stat(country, fatalities_df),
                           }, ignore_index=True)

In [58]:
# First Day Values
def first_day(df, country):
    row = list(df.loc[df.index == country].values[0])
    col = next((i for i, j in enumerate(row) if j.any()), None)     
    first_day = df.columns[col]
    return first_day

# Example:
# print(first_day(top_confirmed_df, "Italy"))
# print(first_day(top_fatalities_df, "Italy"))

In [59]:
# First Day column
firsts = [] 
for country in countries:
    firsts.append(first_day(confirmed_df, country))
all_df['First Day'] = firsts

In [60]:
# Global row
all_df = all_df.append(all_df.sum(), ignore_index=True)
all_df = all_df.replace(all_df.iloc[-1,0], 'Global')
all_df.replace(all_df.iloc[-1, -1], all_df['First Day'].values.min(), inplace=True)
all_df.tail()

,Country_Region,Confirmed,Recovered,Fatalities,First Day
182,Yemen,1,NaN,NaN,4/10/20
183,Zambia,40,28.0,2.0,3/18/20
184,Zimbabwe,14,NaN,3.0,3/20/20
185,Global,1771514,402110.0,108503.0,1/22/20
186,Global,3543028,804236.0,217006.0,1/22/20


In [61]:
# Save JSON
# all_df.to_json('all.json', orient='records')

## Reorganizing Data to First Day of Confirmed Case

In [62]:
# New dataframes with all non-zero values at beginning
def first_day_df(df):
    dates = df.columns.tolist()
    days = range(len(dates))
    
    new_df = pd.DataFrame(columns=days)

    countries = df.index              # list of countries
    for country in countries:
        # Grab the country's row
        row = list(df.loc[df.index == country].values[0])
        # Grab the column index of first non-zero value
        col = next((i for i, j in enumerate(row) if j.any()), None)     
        # Calculate date
        first_day = df.columns[col]
        # Reset first day
        row = row[col:] + row[:col]
        # New rearranged dataframe
        new_df = new_df.append([row])

    # Same countries indexed
    new_df.index = df.index
    new_df = new_df.replace(0,np.NaN)
    
    return new_df

In [63]:
# Confirmed Cases Since First Day
first_confirmed_df = first_day_df(top_confirmed_df)
first_confirmed_df

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
Country_Region,,,,,,,,,,,,,,,,,,,,,
United States,1,1,2,2,5,5,5,5,5,7,...,243762.0,275586.0,308853.0,337072.0,366667.0,396223.0,429052.0,461437.0,496535.0,526396.0
Spain,1,1,1,1,1,1,1,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Italy,2,2,2,2,2,2,2,3,3,3,...,152271.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
France,2,3,3,3,4,5,5,5,6,6,...,90848.0,93773.0,98963.0,110065.0,113959.0,118781.0,125931.0,130727.0,NaN,NaN
Germany,1,4,4,4,5,8,10,12,12,12,...,107663.0,113296.0,118181.0,122171.0,124908.0,NaN,NaN,NaN,NaN,NaN
China,548,643,920,1406,2075,2877,5509,6087,8141,9802,...,82432.0,82511.0,82543.0,82602.0,82665.0,82718.0,82809.0,82883.0,82941.0,83014.0
United Kingdom,2,2,2,2,2,2,2,3,3,3,...,79874.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Iran,2,5,18,28,43,61,95,139,245,388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
first_confirmed_all = first_day_df(confirmed_df)
first_confirmed_all

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,2,10,12.0,23.0,33.0,38.0,42.0,51.0,55.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Algeria,1,1,1.0,1.0,1.0,1.0,3.0,5.0,12.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,1,2,2.0,3.0,3.0,3.0,4.0,4.0,5.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Western Sahara,4,4,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zambia,2,2,2.0,2.0,3.0,3.0,3.0,12.0,16.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
first_fatalities_all = first_day_df(fatalities_df)
first_fatalities_all

,0,1,2,3,4,5,6,7,8,9,...,152,153,154,155,156,157,158,159,160,161
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1.0,1.0,1.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Algeria,1.0,2.0,3.0,4.0,4.0,4.0,7.0,9.0,11.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,1.0,1.0,1.0,1.0,3.0,3.0,3.0,6.0,8.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zambia,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
first_recovered_all = first_day_df(recovered_df)
first_recovered_all

,0,1,2,3,4,5,6,7,8,9,...,152,153,154,155,156,157,158,159,160,161
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,2.0,2.0,2.0,10.0,17.0,17.0,31.0,31.0,33.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Algeria,8.0,8.0,12.0,12.0,12.0,12.0,12.0,32.0,32.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zambia,2.0,2.0,3.0,5.0,7.0,7.0,24.0,25.0,28.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
first_infected_all = first_day_df(infected_df)
first_infected_all

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,2,10,12.0,23.0,33.0,38.0,42.0,51.0,55.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Algeria,1,1,1.0,1.0,1.0,1.0,3.0,5.0,12.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,1,2,2.0,3.0,3.0,3.0,4.0,4.0,5.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yemen,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zambia,2,2,2.0,2.0,3.0,3.0,3.0,12.0,16.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zimbabwe,1,3,3.0,3.0,3.0,3.0,3.0,5.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
first_infected_df = first_day_df(top_infected_df)
first_infected_df

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
Country_Region,,,,,,,,,,,,,,,,,,,,,
China,520,613,884,1367,2026,2819,5408,5967,8006,9588,...,5867.0,5751.0,5597.0,5395.0,5355.0,5308.0,5242.0,5204.0,5150.0,5137.0
France,2,3,3,3,4,5,5,5,6,6,...,75276.0,77424.0,81535.0,90542.0,92507.0,95368.0,100736.0,104064.0,NaN,NaN
Germany,1,4,4,4,5,8,10,12,12,12,...,71582.0,66996.0,65774.0,68258.0,67508.0,NaN,NaN,NaN,NaN,NaN
Iran,2,5,18,28,43,61,95,90,196,315,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Italy,2,2,2,2,2,2,2,3,3,3,...,119737.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Spain,1,1,1,1,1,1,1,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
United States,1,1,2,2,5,5,5,5,5,7,...,234761.0,265879.0,294201.0,319624.0,347086.0,374460.0,405493.0,436027.0,467745.0,495126.0
United Kingdom,2,2,2,2,2,2,2,3,3,3,...,79252.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
